#### Building A Chatbot

In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot what we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics whcih will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11b736c50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11b735990>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Pydi and I am a chief AI Engineer")])

AIMessage(content="As an AI, I have no memory of past conversations and no access to your personal information. So, I don't know your name or what you do. 😊\n\nWould you like to tell me?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 21, 'total_tokens': 66, 'completion_time': 0.081818182, 'prompt_time': 0.000145199, 'queue_time': 0.0131084, 'total_time': 0.081963381}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-09202acb-320d-45aa-8cb1-3371f7b72c2f-0', usage_metadata={'input_tokens': 21, 'output_tokens': 45, 'total_tokens': 66})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(
            content="Hi, My name is Pydi and I am a chief AI Engineer"
        ),
        AIMessage(
            content="Hello Pydi, it's nice to meet you! \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  Do you have any exciting challenges you're tackling?  I'm always eager to learn more about the cutting edge of AI."
        ),
        HumanMessage(
            content="Hey What's my name and what do I do?"
        )
    ]
)

AIMessage(content="You said your name is Pydi, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 108, 'total_tokens': 157, 'completion_time': 0.089090909, 'prompt_time': 0.003331587, 'queue_time': 0.010484232, 'total_time': 0.092422496}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f08284c-4045-4fce-bd8b-ed6deee6924c-0', usage_metadata={'input_tokens': 108, 'output_tokens': 49, 'total_tokens': 157})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history=get_session_history)

In [8]:
config = {"configurable": {"session_id": "chat1"}}

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Pydi and I am a chief AI Engineer")],
    config=config
)

In [13]:
response.content

"Hello Pydi! It's a pleasure to meet you. \n\nBeing a Chief AI Engineer is a very exciting role.  What are some of the most interesting projects you're working on these days? I'm always curious to hear about the cutting edge of AI development. \n\n"

In [14]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Pydi.  \n\nYou introduced yourself at the beginning of our conversation! 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 172, 'total_tokens': 194, 'completion_time': 0.04, 'prompt_time': 0.005533079, 'queue_time': 0.008441409, 'total_time': 0.045533079}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b1fe6ca-37be-4374-b5b0-138660a9417a-0', usage_metadata={'input_tokens': 172, 'output_tokens': 22, 'total_tokens': 194})

In [18]:
## change the config--> session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config = config1
)

In [19]:
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config = config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today?\n"

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config = config1
)
response.content

'Your name is John!  I remember now. 😊  \n\nIs there anything else I can help you with?\n'

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config = config
)
response.content

'Your name is Pydi. 😊 \n\nIs there anything else I can help you with? \n'

#### Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Pydi")]})

AIMessage(content="Hi Pydi, it's nice to meet you!  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 31, 'total_tokens': 59, 'completion_time': 0.050909091, 'prompt_time': 0.000441898, 'queue_time': 0.01386916, 'total_time': 0.051350989}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8c5eb459-1844-411c-a1d4-1494307ebd9c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 28, 'total_tokens': 59})

In [26]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history=get_session_history)


In [27]:
config={"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Pydi")],
    config=config
)

In [28]:
response.content

"Hello Pydi! It's nice to meet you. \n\nWhat can I do for you today?  😄 \n"

In [39]:
## Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",

        ),
        MessagesPlaceholder(variable_name="messages"),

    ]
)

chain = prompt | model 

In [40]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Pydi")],"language":"Telugu"})
response.content

'నమస్తే, ప్యడి! 😊 \n\nనాకు మీ పేరు తెలుసు. మీరు ఏం అడిగితే, నేను మీకు సహాయం చేయడానికి సిద్ధంగా ఉన్నాను. \n\nఏమి అడిగితే చెప్పండి! \n\n'

Let's now wrap this more complicated chian in a Message History class. This time, because there are mulitple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [32]:
config={"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Pydi")],"language":"Telugu"},
    config=config

)
response.content

'నమస్తే Pydi! 😊\n\nమేము మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాము. ఏమి అడుగువా? \n'

In [33]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Whats my name?")],"language":"Telugu"},
    config=config

)

In [34]:
response.content

'మీ పేరు Pydi. 😊 \n'

#### Managing the conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM.Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [41]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good asistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good asistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")
                               | trimmer)
                               | prompt
                               | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do I like?")],
        "language": "English"
    }
    
)
response.content


"As an AI, I don't have access to your personal information, like your favorite ice cream flavor. \n\nWhat's your favorite flavor? Maybe I can tell you some fun facts about it! 😊🍦\n"

In [45]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem did i ask")],
        "language": "English"
    }
    
)
response.content

'You asked me "What is 2+2?".\n'

In [46]:
## Lets wrap this in the message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "chat5"}}

In [47]:
response = with_message_history.invoke(
    {'messages': messages + [HumanMessage(content="Whats my name?")],"language":"Telugu"},
    config=config

)
response.content

"I don't have access to any past conversations or personal information about you, so I don't know your name. 😊 \n\nWould you like to tell me your name?  \n"

In [49]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What math problem did i ask?")],
        "language": "English"
    },
    config=config
    
)
response.content

"As a large language model, I have no memory of past conversations. So I don't know what math problem you asked me.\n\nIf you have a math problem you'd like me to help with, please ask! I'm ready. 😊  \n\n"